#  Setup stuff: Connectivity

In [28]:
#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [29]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [30]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [43]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [44]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, GgiD32tpHD4dkRjywMZZUiccEjsBxGVq4nNGSgcz7qKF, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [33]:
import pandas as pd

print("--")

--


In [34]:
#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

df_airports.head(20)


,airport_code,airport_name,LABEL
0,MKE,Milwaukee,Airport
1,ORD,Chicago O-Hare,Airport
2,SJC,San Jose,Airport
3,DEN,Denver,Airport


In [35]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(30)


,START_ID,END_ID,DISTANCE,NUM_HOPS,TYPE
0,MKE,ORD,66,1,FLIES_TO
1,ORD,MKE,66,1,FLIES_TO
2,ORD,DEN,886,1,FLIES_TO
3,DEN,ORD,886,1,FLIES_TO
4,SJC,DEN,948,1,FLIES_TO
5,DEN,SJC,948,1,FLIES_TO
6,SJC,ORD,1829,1,FLIES_TO


In [45]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

          0/? [?op/s]

--


In [ ]:
# ##################################################################

In [46]:

#  Add a second node type, and related edge
#

l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code     = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


          0/? [?op/s]

          0/? [?op/s]

In [ ]:
# ##################################################################

In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


In [47]:

#  What's in the vertices/nodes ..

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+------------------+----------+---------------------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_code   | n.airport_name   | n.type   | n.restaurant_code   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+------------------+----------+---------------------+-----------------------------|
|  0 |               0 | ['Airport']    | Airport   | MKE              | Milwaukee        | node     | nan                 | nan                         |
|  1 |               2 | ['Restaurant'] | nan       | nan              | nan              | node     | PAP                 | Pappadeauxs Seafood Kitchen |
|  2 |               1 | ['Airport']    | Airport   | DEN              | Denver           | node     | nan                 | nan                         |
|  3 | 562949953421312 | ['Airport']    | Airport   | ORD             

In [50]:

#  What's in the edge .. 

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   RETURN r
   // RETURN  n.id AS FROM, m.id AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label    |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   | r.DRIVABLE   |
|----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------|
|  0 | 562949953421312 | ['Airport']    |               1 | FLIES_TO   |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
|  1 | 562949953421313 | ['Airport']    |               0 | FLIES_TO   |          948 |            1 | FLIES_TO |               1 | ['Airport']      | edge     | nan          |
|  2 | 562949953421312 | ['Airport']    |               2 | FLIES_TO   |          886 |            1 | FLIES_TO |  

In [53]:

#  Full graph

l_result = my_graph.query("""
   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   """, contextualize=True)


print(l_result)
# print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

          0/? [?op/s]

     n.internal_id   n.labels  n.LABEL n.airport_code  n.airport_name n.type    m.internal_id      m.labels  m.LABEL m.airport_code  ...  r.end.labels    r.internal_id     r.label  r.DISTANCE r.NUM_HOPS    r.TYPE       r.start.id r.start.labels  r.type r.DRIVABLE
0                1  [Airport]  Airport            DEN          Denver   node  562949953421313     [Airport]  Airport            SJC  ...     [Airport]                0    FLIES_TO       948.0        1.0  FLIES_TO                1      [Airport]    edge        NaN
1                1  [Airport]  Airport            DEN          Denver   node  562949953421312     [Airport]  Airport            ORD  ...     [Airport]                2    FLIES_TO       886.0        1.0  FLIES_TO                1      [Airport]    edge        NaN
2                0  [Airport]  Airport            MKE       Milwaukee   node  562949953421312     [Airport]  Airport            ORD  ...     [Airport]                1    FLIES_TO        66.0        1.0  FLIE